# Multi Class Classification with PyTorch and Custom NN

In [19]:
# iris dataset (well known and used in ml)
# classification problem with 4 features, 3 classes and 150 data

# features are numericals, with output class 
# given petal and sepal length, width predict the class 

# Create a NN that outputs 3 probability distribution

# The dataset can be easily loaded from sklearn 
from sklearn.datasets import load_iris

import torch
import torch.nn as nn
import torch.optim as optim 
import pandas as pd 
import numpy as np 

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [2]:
iris = load_iris()

df = pd.DataFrame(iris.data, columns=iris.feature_names)
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [3]:
df['target'] = iris.target # add also the class label

df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [4]:
# split dataset 
# use stratify to specify the criterion of split 
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['target']) # based on what 


In [ ]:
# Save train and test in X,y variables|
X_train, y_train = train_df.drop(columns=['target']), train_df['target']
X_test, y_test = test_df.drop(columns=['target']), test_df['target']

In [6]:
# Scale dataset
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [8]:
# Convert to tensor to use it in NN 
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long) # torch.long is used for classification
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

In [9]:
# Creating the NN model 
class IrisClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(IrisClassifier, self).__init__()

        # crate with sequential (simple Net)
        self.network = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        return self.network(x)

In [ ]:
# Define useful variables as network dimensions
input_dim = X_train.shape[1]
hidden_dim = 16
output_dim = 3 # number of classes

In [16]:
model = IrisClassifier(input_dim, hidden_dim, output_dim)

# Define additional network components 
criterion = nn.CrossEntropyLoss() # In classification cross entropy is used 
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [17]:
# Training the model 
epochs = 500

for epoch in range(epochs):
    model.train()         # set model in train mode
    optimizer.zero_grad() # clean gradients 

    # forward pass (full data)
    predictions = model(X_train_tensor)
    loss = criterion(predictions, y_train_tensor)
    loss.backward() # backward propagation

    optimizer.step() # step of optimization updatig weights

    if (epoch + 1) % 50 == 0: 
        print(f"Epoch [{epoch+1}/epochs], Loss: {loss.item():.4f}")

Epoch [50/epochs], Loss: 0.0937
Epoch [100/epochs], Loss: 0.0369
Epoch [150/epochs], Loss: 0.0240
Epoch [200/epochs], Loss: 0.0115
Epoch [250/epochs], Loss: 0.0043
Epoch [300/epochs], Loss: 0.0021
Epoch [350/epochs], Loss: 0.0012
Epoch [400/epochs], Loss: 0.0008
Epoch [450/epochs], Loss: 0.0006
Epoch [500/epochs], Loss: 0.0004


In [18]:
# see performance of the model 
model.eval() # set in evaluation mode
with torch.no_grad(): # tell tensor to avoid gradient update 
    y_pred = model(X_test_tensor)
    y_pred_labels = torch.argmax(y_pred, dim=1) # take the highest probability of the prediction 

    # compare test
    accuracy = (y_pred_labels == y_test_tensor).sum().item() / y_test_tensor.size(0) # number of correct prediction / number of values  = TP 
    print(f"Test accuracy : {accuracy:.4f}")

    

Test accuracy : 0.9667


In [26]:
# Creat a function to do inference on a single row 
def predict_iris(sepal_length, sepal_width, petal_length, petal_width):
    input_data = np.array([[sepal_length, sepal_width, petal_length, petal_width]])
    input_data = scaler.transform(input_data)

    # convert to tensor 
    input_data_tensor = torch.tensor(input_data, dtype=torch.float32)

    model.eval()
    with torch.no_grad(): # tell tensor to avoid gradient update 
        prediction = model(input_data_tensor)
        predicted_class = torch.argmax(prediction, dim=1).item() # take the highest probability of the prediction  

    return iris.target_names[predicted_class]

In [29]:
# Use the defined model predictor function 

prediction = predict_iris(5.1, 3.5, 1.4, 0.2)
print(f"Predicted Species : {prediction}")

Predicted Species : setosa


/home/ale/anaconda3/envs/pytorch_gpu12_8/lib/python3.11/site-packages/sklearn/utils/validation.py:2691: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
